In [ ]:
import reactionmodel.load
import glob
import os
import re
import numpy as np
import pandas as pd
from dataclasses import dataclass
from typing import NamedTuple
from reactionmodel.specification import SimulationSpecification

from simulators import SIMULATORS

@dataclass
class SimulatorArguments():
    t_span: tuple
    sample_points: tuple

TEST_ARGUMENTS = SimulatorArguments((0.0, 50.0), np.linspace(0, 50, 51))

inital_cwd = os.getcwd()

In [ ]:
os.chdir(inital_cwd)
test_dir = "./tests/sbml-tests/"
tests = glob.glob(os.path.join(test_dir, 'sbml-*'))

target_test = "sbml-001-birth-death"
target_check = "p01ic01"
target_catchment = []

specifications = {}

for test in tests:
    if os.path.basename(test) == target_test:
        run_tests_from_dir(test)

In [ ]:
def do_simulations(s, n=1000, config_path=''):
    results = []
    simulator = s.simulator
    forward_time = SIMULATORS[simulator]
    rng = np.random.default_rng()
    initial_condition = s.model.make_initial_condition(s.initial_condition)
    simulation_options = s.simulation_options.copy()

    if simulator == 'hybrid':
        import hybrid
        partition_path = simulation_options.pop('partition')
        print(os.path.join(config_path, partition_path))
        partition_scheme = hybrid.load_partition_scheme(os.path.join(config_path, partition_path))
        simulation_options['partition_function'] = partition_scheme.partition_function
    s.model.bake_k(parameters=s.parameters, jit=True)
    for i in range(n):
        print(i)
        result = forward_time(initial_condition, TEST_ARGUMENTS.t_span, s.model.k_jit, s.model.stoichiometry(), s.model.rate_involvement(), rng, discontinuities=TEST_ARGUMENTS.sample_points, **simulation_options)
        results.append(result)
    return results

def run_tests_with_checks(root, specifications):
    tests_to_do = []
    for check in glob.glob(os.path.join(root, 'checks', '*/')):
        check_dir_root = check.split('/')[-2]
        if target_check is not None:
            if check_dir_root != target_check:
                print(f"Since we are targeting a single check, we are skipping {check_dir_root}")
                continue
        tests_to_do.append((specifications[check_dir_root], check))
    print(f"Performing tests with benchmark data for {len(tests_to_do)}/{len(specifications)} combinations.")

    for spec_tuple,check_dir in tests_to_do:
        specification, config_dir = spec_tuple
        check_data = pd.read_csv(glob.glob(os.path.join(check_dir, '*.csv'))[0])
        results = do_simulations(specification, config_path=config_dir)
        desired_species = set([c.split('-')[0] for c in check_data.columns if len(c.split('-')) > 1])
        assert len(desired_species) == 1
        desired_species = list(desired_species)[0]
        target_index = [s.name for s in specification.model.species].index(desired_species)
        print("TARGET INDEX: ", target_index)
        aligned = align_results(results, check_data['time'], target_index, desired_species)
        if target_check:
            print("Catching our targeted check in target_catchment")
            target_catchment.append((aligned, check_data))
        return aligned

def align_results(results, time, target_index, species_name):
    all_aligned = []
    for r in results:
        aligned = []
        t_history = r.t_history
        for t in time[:-1]:
            idx = np.argmin(np.abs(t-t_history))
            aligned.append((r.t_history[idx], r.y_history[target_index,idx]))
        aligned.append((r.t, r.y[target_index]))
        all_aligned.append(pd.DataFrame.from_records(aligned, columns=['time', species_name]))
    return all_aligned

def get_files(root, individual, collection, pattern):
    if os.path.isfile(os.path.join(root, individual)):
        return [os.path.join(root, individual)]
    return glob.glob(os.path.join(root, collection, pattern))

class SpecTuple(NamedTuple):
    specification: SimulationSpecification
    config_path: str

def run_tests_from_dir(dir):
    model_paths  = get_files(dir, 'model.txt', 'models', 'model*.txt')
    params_paths = get_files(dir, 'parameters.txt', 'parameters', 'parameters*.txt')
    config_paths = get_files(dir, 'config.txt', 'configurations', 'config*.txt')
    ic_paths     = get_files(dir, 'ic.txt', 'initial_conditions', 'initial*.txt')
    for model_path in model_paths:
        for params_path in params_paths:
            for config_path in config_paths:
                for ic_path in ic_paths:
                    specification = reactionmodel.load.load_specification(model_path, params_path, config_path, ic_path)
                    # use the parameter and ic file names as a unique identifier for this combination
                    # later, we will look up all the combinations that we have test data for, and run simulations to check
                    model_match = re.search('[a-z]+([0-9]+)\.txt', model_path)
                    config_match = re.search('[a-z]+([0-9]+)\.txt', config_path)
                    param_match = re.search('[a-z]+([0-9]+)\.txt', params_path)
                    ic_match = re.search('[a-z]+([0-9]+)\.txt', ic_path)
                    matches = [('m', model_match), ('c', config_match), ('p', param_match), ('ic', ic_match)]
                    identifier = ''
                    for id_str, match in matches:
                        if match:
                            identifier += id_str + str(match[1])
                    # if identifier == '': all of the configuration files lived in root directory, so the check should just live in the root of the check directory
                    specifications[identifier] = SpecTuple(specification, os.path.dirname(config_path))
    run_tests_with_checks(dir, specifications)

# Analyze results

In [ ]:
single_target = target_catchment[0]
results, check = single_target

In [ ]:
indexed_results = []
for r in results:
    r['time'] = np.round(r['time'], 5)
    r = r.set_index('time')
    indexed_results.append(r)

In [ ]:
results_to_check = pd.concat([pd.concat(indexed_results, axis=1).mean(axis=1), pd.concat(indexed_results, axis=1).std(axis=1)], axis=1)
results_to_check.columns=['mean', 'std']
results_to_check

In [ ]:
# https://github.com/sbmlteam/sbml-test-suite/blob/release/cases/stochastic/DSMTS-userguide-31v2.pdf
z_t = (results_to_check['mean'] - check['X-mean'])/(check['X-sd']) * np.sqrt(1000)

In [ ]:
z_t